## III. Imputation and Encoding


### Get data and desired features

In [13]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('./data/data_built_features.csv')

In [6]:
for col in data.columns.values:
    print col

dba_name
facility_type
inspection_date
inspection_type
inspection_id
violations
latitude
license_
longitude
results
risk
zip
police_district
precinct
account_number
site_number
ward_precinct
business_activity_Consumption of Liquor on Premises
business_activity_Preparation of Food and Dining on Premise With Seating
business_activity_Retail Sale of Tobacco
business_activity_Retail Sales of Packaged Liquor
business_activity_Retail Sales of Perishable Foods
creation_date
num_sanitation_in_police_dist
index
district
crime_count_by_district
date
near_x
near_y
point_crime_count
point_sanit_count
result_binary
grocery
Canvass
License
Canvass Re-Inspection
Complaint
License Re-Inspection
Short Form Complaint
Complaint Re-Inspection
Suspected Food Poisoning
Consultation
License-Task Force
TMAX
TMAX_3DayAvg
TMAX_10DayAvg
TMAX_30DayAvg
violations_count
citations_count
critical_count
serious_count
minor_count
corrected_count
previous_count
previous_fraction
previous_result
time_since_last_inspectio

In [7]:
cols = ['inspection_date',
        'facility_type',
        'latitude',
        'longitude',
        'results',
        'risk',
        'business_activity_Consumption of Liquor on Premises',
        'business_activity_Preparation of Food and Dining on Premise With Seating',
        'business_activity_Retail Sale of Tobacco',
        'business_activity_Retail Sales of Packaged Liquor',
        'business_activity_Retail Sales of Perishable Foods',
        'Canvass', # inspection_types
        'License',
        'Canvass Re-Inspection',
        'Complaint',
        'License Re-Inspection',
        'Short Form Complaint',
        'Complaint Re-Inspection',
        'Suspected Food Poisoning',
        'Consultation',
        'License-Task Force',
        'point_crime_count',
        'point_sanit_count',
        'result_binary',
        'TMAX',
        'TMAX_3DayAvg',
        'TMAX_10DayAvg',
        'TMAX_30DayAvg',
        'previous_count',
        'previous_fraction',
        'previous_result',
        'time_since_last_inspection',
        'previous_violations',
        'previous_citations',
        'previous_critical',
        'previous_serious',
        'previous_minor',
        'previous_corrected']

print data[cols].shape
print data.shape

# select specific features
data_sub = data[cols]
# what would happen if we dropped NaNs
print data_sub.dropna(axis=0, how='any').shape

(120308, 38)
(120308, 64)
(61493, 38)


In [8]:
data_sub = data_sub.copy()

In [9]:
# temporary -- dumping correct crime data and sanitation complaint data into dataset
data_dump = pd.read_csv('./data/crime_sanit.csv')

In [10]:
data_sub['crime'] = data_dump['crime'].copy()
data_sub['sanit'] = data_dump['sanitation'].copy()

In [11]:
# convert string to datetime to be safe
data_sub['inspection_date'] = pd.to_datetime(data_sub['inspection_date'])

print data_sub['inspection_date'].min()
print data_sub['inspection_date'].max()

2010-01-04 00:00:00
2016-12-02 00:00:00


### Imputation and encoding 

In [21]:
# non-KNN approach:
# replace null values with feature mean (quantitative) or feature mode (qualitative)
# for column in data_sub.columns:
#     if (data_sub[column].unique().shape[0] < 8) or (data_sub[column].dtype == np.dtype('object')):
#         data_sub[column].fillna(value = data_sub[column].mode()[0], inplace = True)
#     else:
#         data_sub[column].fillna(value = data_sub[column].mean(), inplace = True)

# check no null values
nan_cnt = 0
for i, column in enumerate(data_sub.columns):
    print i, column, np.array(data_sub[column].isnull()).sum()
    nan_cnt = nan_cnt + np.array(data_sub[column].isnull()).sum()

print '\n\n'
print 'Total cells in dataset:', data_sub.size
print 'Total cells containing NaN in dataset:', nan_cnt
print 'Ratio of NaN to total datapoints:', round(nan_cnt / float(data_sub.size),2)

0 inspection_date 0
1 facility_type 640
2 latitude 460
3 longitude 460
4 results 0
5 risk 38
6 business_activity_Consumption of Liquor on Premises 7205
7 business_activity_Preparation of Food and Dining on Premise With Seating 7205
8 business_activity_Retail Sale of Tobacco 7205
9 business_activity_Retail Sales of Packaged Liquor 7205
10 business_activity_Retail Sales of Perishable Foods 7205
11 Canvass 0
12 License 0
13 Canvass Re-Inspection 0
14 Complaint 0
15 License Re-Inspection 0
16 Short Form Complaint 0
17 Complaint Re-Inspection 0
18 Suspected Food Poisoning 0
19 Consultation 0
20 License-Task Force 0
21 point_crime_count 2208
22 point_sanit_count 18266
23 result_binary 0
24 TMAX 17440
25 TMAX_3DayAvg 17440
26 TMAX_10DayAvg 17732
27 TMAX_30DayAvg 18690
28 previous_count 0
29 previous_fraction 26924
30 previous_result 26924
31 time_since_last_inspection 26924
32 previous_violations 33687
33 previous_citations 33687
34 previous_critical 33687
35 previous_serious 33687
36 previou

In [ ]:
# how should we handle the historical count data?
# there's no optimal solution: either (1) impute, biasing that they wouldn't have done well 
# or (2) set to 0, biasing that they would've done well? We'll set to 0 for simplicity.

print 'count of obs for which previous_count == 0:', data_sub[data_sub['previous_count'] == 0.0].shape[0]
print 'this corresponds with number of nans for previous_fraction, previous_result, time_since_last_inspection, which is what we\'d expect'

# set all these to 0 then
hist_cols = ['previous_fraction',
            'previous_result',
            'time_since_last_inspection',
            'previous_violations',
            'previous_citations',
            'previous_critical',
            'previous_serious',
            'previous_minor',
            'previous_corrected']

for hist_col in hist_cols:
    data_sub.loc[data_sub['previous_count'] == 0.0, hist_col] = data_sub.loc[data_sub['previous_count'] == 0.0, hist_col].fillna(value = 0)
    
print '\n here are NaN counts after this step: \n'
for i, column in enumerate(data_sub.columns):
    print i, column, np.array(data_sub[column].isnull()).sum()
    
print '37-42 still have a uniform number of null values because they had no text from the inspection, so nothing to scrape'

In [ ]:
# obs with no null values
print 'count of obs with no null values:', data_sub.shape[0] - data_sub.isnull().any(axis = 1).sum()
print 'count of features with no null values:', data_sub.shape[1] - data_sub.isnull().any(axis = 0).sum()

In [ ]:
# features with no null values we can use to impute
data_sub.isnull().any(axis = 0)[data_sub.isnull().any(axis = 0) == False].index.values

In [ ]:
# so that the imputer has enough predictors, to work with, fill in risk by mean / mode approach since only 63 obs have nulls

data_sub['risk'].fillna(value = data_sub['risk'].mode()[0], inplace = True)

# same
# data_sub['inspection_type'].fillna(value = data_sub['inspection_type'].mode()[0], inplace = True)

# check
print 'number of null values in risk:', data_sub['risk'].isnull().sum()
# print 'number of null values in inspection_type:', data_sub['inspection_type'].isnull().sum()

In [ ]:
# KNN 
from sklearn.neighbors import NearestNeighbors

In [ ]:
# have to encode the data first... but facility_type is annoying to do...
print data_sub.facility_type.value_counts()[0:10]

take_types = data_sub.facility_type.value_counts()[0:50].index.values

In [ ]:
for i, row in data_sub[['facility_type']].iterrows():
    if row.values not in take_types:
        data_sub.loc[i, 'facility_type'] = 'Other'
    else:
        continue
    if (i % 500 == 0):
        print 'finished iteration:', i

In [ ]:
# check response
print 'Useless response count:', sum((data_sub.results != 'Pass') & (data_sub.results != 'Pass w/ Conditions') & (data_sub.results != 'Fail'))
indices = (data_sub.results == 'Pass') | (data_sub.results == 'Pass w/ Conditions') | (data_sub.results == 'Fail')
data_sub = data_sub[indices]

In [ ]:
# risk
indices = (data_sub.risk == 'Risk 1 (High)') | (data_sub.risk == 'Risk 2 (Medium)') | (data_sub.risk == 'Risk 3 (Low)')
data_sub = data_sub[indices]

In [ ]:
data_sub = data_sub.reset_index(drop=True)

In [ ]:
# have to these encode first
data_sub_encoded = pd.DataFrame({})
for column in data_sub.columns:
#     if (data_sub[column].unique().shape[0] < 8) or data_sub[column].dtype == np.dtype('object'):
    if (data_sub[column].dtype == np.dtype('object')) & (column not in ['results']):
        encoding = pd.get_dummies(data_sub[column])
        data_sub_encoded = pd.concat([data_sub_encoded, encoding], axis = 1)
    else:
#         data_sub_encoded = pd.concat([data_sub_encoded, data_sub[[column]].astype(float)], axis = 1)
        data_sub_encoded = pd.concat([data_sub_encoded, data_sub[[column]]], axis = 1)

In [ ]:
# train data are observations that are complete
# test data (which we wish to impute) are observations that are incomplete
train = data_sub_encoded[np.logical_not(data_sub_encoded.isnull().any(axis = 1).values)]
test = data_sub_encoded[data_sub_encoded.isnull().any(axis = 1).values]

In [ ]:
# reset indices because we've dropped rows, NearestNeighbors was returning wrong indices because they weren't reset
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# dont actually have to split x and y since we specify what features we're imputing on in impute_cols
# x_train = train.drop(['result_binary', 'results'], axis = 1) 
# x_test = test.drop(['result_binary', 'results'], axis = 1) 

In [ ]:
data_sub.isnull().any(axis = 0)[data_sub.isnull().any(axis = 0) == False].index.values

In [ ]:
impute_cols = ['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)', 'previous_count', 'previous_fraction',
       'previous_result', 'time_since_last_inspection']

In [ ]:
knn = NearestNeighbors(n_neighbors = 5)
knn.fit(train[impute_cols]) 

In [ ]:
# return k nearest neighbors based on features we have for all observations
# then fill in other values using these nearest neighbors 

import math
from scipy import stats

for i, row in test.iterrows(): # for each observation with incomplete features
    ind = knn.kneighbors(X = row[impute_cols].values.reshape(1,-1), return_distance=False)[0] # return indices of nearest neighbors with complete features
    for col in test.columns.values[row.isnull().values]: # for each feature that're NaNs    
        train_vals = np.array(train.loc[ind, col]) # get vals from nearest neighbors for this col
        if train[col].unique().shape[0] <= 2: # if indicator 
            test.loc[i, col] = stats.mode(train_vals)[0][0] # fill w/ mode
        else:
            test.loc[i, col] = train_vals.mean() # fill w/ mean
    if (i % 500 == 0):
        print 'finished iteration:', i

In [ ]:
# this won't work, apply flattens the thing
# import math
# from scipy import stats

# def impute_row(row):
#     ind = knn.kneighbors(X = row[impute_cols].values.reshape(1, -1), return_distance=False)[0]
#     for col in test.columns.values[row.isnull().values]: # for each feature that're NaNs    
#         train_vals = np.array(train.loc[ind, col])
#         if train[col].unique().shape[0] <= 2:
#             row[col] = stats.mode(train_vals)[0][0]
#         else:
#             row[col] = train_vals.mean()

# test = test.apply(impute_row, axis = 1)


In [ ]:
data_ready = pd.concat([train, test], axis = 0)
data_ready = data_ready.reset_index(drop=True)

In [ ]:
# so don't have to rerun the code above
data_ready.to_csv('./data/data_ready.csv')